# Best trial for Helium 

# The optimization of the parameters 

In [2]:
import tensorflow as tf # for implementing optimization
import numpy as np 
import matplotlib.pyplot as plt 


import parameters as pr  
import functions_module as mc 



2023-02-05 06:41:13.401730: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-05 06:41:13.553729: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-05 06:41:13.553745: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-05 06:41:14.292924: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
import sympy as sym 
from sympy import symbols, diff, lambdify 
from numba import jit 
from scipy.optimize import minimize

import re 
import pandas as pd 

**Testattu**
1. metropolisSamplingFunction ja monteCarloIntegrationFunction, joilla on parametrit,  voivat korvata vanhemmat vastaavat funktiot. 

# Global values 

In [4]:
historyTable = {'Energy': [], 'Variance':[], 'Std error of mean': [], 'parameters': []}

def saveToHistoryTable(energyAndErrors: tuple or list, params: float or list or tuple): 
    '''
    Save integration results to the table historyTable (global value)
    
    - energyAndErrors: energy and their errors in format (energy, variance, std error of mean)
    - parameters: parameters.
        - it is float-type if there is only one parameter 
        - it is list or tuple-type if there are more than one parameter
    '''
    
    
    historyTable['Energy'].append(energyAndErrors[0])
    historyTable['Variance'].append(energyAndErrors[1])
    historyTable['Std error of mean'].append(energyAndErrors[2])
    historyTable['parameters'].append(params)

# Algorithm

In [5]:

@jit(nopython = True )
def probabilityFunction(x: np.ndarray, params: list): 
    particleOnexyz = x[0] 
    particleTwoxyz = x[1]
    return (
        (1 + params[1]*np.sqrt((particleOnexyz[0] - particleTwoxyz[0])**2 + (particleOnexyz[1] - particleTwoxyz[1])**2 + (particleOnexyz[2] - particleTwoxyz[2])**2))**2/
     -  np.exp(2*params[0]*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)))
        
    )





def localEnergyFunction(x: np.ndarray, params: list ): 
    particleOnexyz = x[0] 
    particleTwoxyz = x[1]
    

    
    
    return (  
                    (np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -    2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -     np.sqrt(particleOnexyz[0]**2 - 2*particleOnexyz[0]*particleTwoxyz[0] + particleTwoxyz[0]**2 + particleOnexyz[1]**2 - 2*particleOnexyz[1]*particleTwoxyz[1] + particleTwoxyz[1]**2 + particleOnexyz[2]**2 - 
         -       2*particleOnexyz[2]*particleTwoxyz[2] + particleTwoxyz[2]**2) - 
         -    2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)*
         -     np.sqrt(particleOnexyz[0]**2 - 2*particleOnexyz[0]*particleTwoxyz[0] + particleTwoxyz[0]**2 + particleOnexyz[1]**2 - 2*particleOnexyz[1]*particleTwoxyz[1] + particleTwoxyz[1]**2 + particleOnexyz[2]**2 - 
         -       2*particleOnexyz[2]*particleTwoxyz[2] + particleTwoxyz[2]**2) - 
         -    params[0]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)*
         -     np.sqrt(particleOnexyz[0]**2 - 2*particleOnexyz[0]*particleTwoxyz[0] + particleTwoxyz[0]**2 + particleOnexyz[1]**2 - 2*particleOnexyz[1]*particleTwoxyz[1] + particleTwoxyz[1]**2 + particleOnexyz[2]**2 - 
         -       2*particleOnexyz[2]*particleTwoxyz[2] + particleTwoxyz[2]**2) + 
         -    params[0]*np.sqrt(particleOnexyz[0]**2 - 2*particleOnexyz[0]*particleTwoxyz[0] + particleTwoxyz[0]**2 + particleOnexyz[1]**2 - 2*particleOnexyz[1]*particleTwoxyz[1] + particleTwoxyz[1]**2 + particleOnexyz[2]**2 - 
         -       2*particleOnexyz[2]*particleTwoxyz[2] + particleTwoxyz[2]**2)*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 
         -       np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)) - 
         -    params[1]*(2*particleOnexyz[1]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) - 
         -       params[0]*particleOnexyz[1]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) - 
         -       4*particleOnexyz[1]*particleTwoxyz[1]*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 
         -       3*params[0]*particleOnexyz[1]*particleTwoxyz[1]*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 
         -       2*particleTwoxyz[1]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) - 
         -       2*params[0]*particleTwoxyz[1]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 
         -       2*particleOnexyz[2]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) - 
         -       params[0]*particleOnexyz[2]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) - 
         -       4*particleOnexyz[2]*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*particleTwoxyz[2] + 
         -       3*params[0]*particleOnexyz[2]*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*particleTwoxyz[2] + 
         -       2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*particleTwoxyz[2]**2 - 
         -       2*params[0]*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*particleTwoxyz[2]**2 + 
         -       2*particleOnexyz[1]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       2*params[0]*particleOnexyz[1]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       4*particleOnexyz[1]*particleTwoxyz[1]*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       3*params[0]*particleOnexyz[1]*particleTwoxyz[1]*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       2*particleTwoxyz[1]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       params[0]*particleTwoxyz[1]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       2*particleOnexyz[2]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       2*params[0]*particleOnexyz[2]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       params[0]**2*particleOnexyz[1]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -        np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       2*params[0]**2*particleOnexyz[1]*particleTwoxyz[1]*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -        np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       params[0]**2*particleTwoxyz[1]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -        np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       params[0]**2*particleOnexyz[2]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -        np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       4*particleOnexyz[2]*particleTwoxyz[2]*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       3*params[0]*particleOnexyz[2]*particleTwoxyz[2]*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       2*params[0]**2*particleOnexyz[2]*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*particleTwoxyz[2]*
         -        np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       2*particleTwoxyz[2]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       params[0]*particleTwoxyz[2]**2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -       params[0]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*particleTwoxyz[2]**2*
         -        np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -       np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)*
         -        np.sqrt(particleOnexyz[0]**2 - 2*particleOnexyz[0]*particleTwoxyz[0] + particleTwoxyz[0]**2 + particleOnexyz[1]**2 - 2*particleOnexyz[1]*particleTwoxyz[1] + particleTwoxyz[1]**2 + particleOnexyz[2]**2 - 
         -          2*particleOnexyz[2]*particleTwoxyz[2] + particleTwoxyz[2]**2) + 
         -       particleOnexyz[0]*particleTwoxyz[0]*(-2*params[0]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -           np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) - 
         -          4*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 
         -             np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)) + 
         -          3*params[0]*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2))
         -        0 ) + particleTwoxyz[0]**2*(params[0]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -           np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -          2*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 
         -             np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)) - 
         -          params[0]*(2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2))
         -         0 ) + particleOnexyz[0]**2*(params[0]**2*np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -           np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 
         -          2*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 
         -             np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)) - 
         -          params[0]*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 2*np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2))
         -         0 )))/
         -  (np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2)*
         -    (1 + params[1]*np.sqrt((particleOnexyz[0] - particleTwoxyz[0])**2 + (particleOnexyz[1] - particleTwoxyz[1])**2 + (particleOnexyz[2] - particleTwoxyz[2])**2))*
         -    np.sqrt((particleOnexyz[0] - particleTwoxyz[0])**2 + (particleOnexyz[1] - particleTwoxyz[1])**2 + (particleOnexyz[2] - particleTwoxyz[2])**2)*
         -    np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2))
    
    )
  







In [6]:

# MUST BE FLOAT!!!
alpha = 2.0   # initial guess of the parameter
beta = 2.0 
paramsValuesList = [alpha, beta]       # list of parameters 
paramsVariableList = [sym.Symbol('a'), sym.Symbol('b')] # list of parameters as symbolic variables 



patience = 1 # how many times we accept the energy to jump to higher value than the previous energy 


In [ ]:

loopMark = 0 # which loop we are going on 

while patience >= 0: 

    configSamples_Metropolis = mc.metropolisSamplingFunction(coordinateValueRange = (-1, 1), 
                                                         numberOfParticles = 2, 
                                                         numberOfConfig = 10000, 
                                                         probabilityFunction = probabilityFunction, 
                                                        numberOfIterations= 100000, 
                                                          params =  paramsValuesList
                                                         ) 



    # integration approximation using current parameter value 
    integrationResultAsValue = mc.monteCarloIntegrationFunction(configSamples_Metropolis, localEnergyFunction = localEnergyFunction,  params= paramsValuesList, needError = True)
    # save the results to the table 
    saveToHistoryTable(integrationResultAsValue, paramsValuesList)



    # integration approximation with parameter as variable for optimization 
    # return tuple, where first is the integration result with parameters as variables
    integrationResultWithParameter = mc.monteCarloIntegrationFunction(configSamples_Metropolis, localEnergyFunction = localEnergyFunction,  params= paramsVariableList)




    
    
    # optimization: scipy implementation 
    def func(y): 
        y1_float, y2_float  = y 
        return integrationResultWithParameter[0].subs([(paramsVariableList[0], y1_float),(paramsVariableList[1], y2_float) ])
    
    
    solution = minimize(func, paramsValuesList, method = 'SLSQP')  # minimize using current parameter values 
    
    paramsValuesList = list(solution['x'])  # get the optimized parameters list and update paramsValuesList (global value)

    
    # transfrom to python float. Avoid issues with sympy.float and numpy.ufunc incompatibility 
    paramsValuesList = list(map(float, paramsValuesList))   

             
    
    # when we have at least two energy values in the table, we can compare them 
    if loopMark >= 1: 
        # if current calculated energy is bigger than energy calculated in the previous loop, reduce patience by 1
        if historyTable['Energy'][-1] > historyTable['Energy'][-2]: 
            patience -= 1
    
    print(loopMark) 
    
    loopMark += 1 
    break 

/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/monteCarloIntegration.py:63: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'params' of function 'probabilityFunction'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../tmp/ipykernel_3709/120527228.py", line 1:
<source missing, REPL/exec in use?>

  w = probabilityFunction(  trialConfiguration, params  )/probabilityFunction(currentConfig, params)
/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'params' of function 'metropolisSamplingFunction'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-f

In [ ]:
historyTable

In [19]:
type(integrationResultWithParameter[0])

sympy.core.add.Add

# Testaus 

## Regex

In [27]:
string = '-1.0*a**2 + 3.98813746927895*a - 6.72856476573052'
re.findall(r'[-]?\d+[.]{1}\d+', string.replace(' ', ''))


['-1.0', '3.98813746927895', '-6.72856476573052']